In [53]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, callbacks
from tensorflow.keras.optimizers import SGD, Adagrad, Adam
from tensorflow.keras.initializers import RandomNormal, GlorotUniform, HeUniform
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [24]:
reviews_df = pd.read_csv('https://raw.githubusercontent.com/selvinfurtado01/schulich_data_science/refs/heads/main/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv')

In [25]:
reviews_df.head(10)

,Unnamed: 0,recipe_number,recipe_code,recipe_name,comment_id,user_id,user_name,user_reputation,created_at,reply_count,thumbs_up,thumbs_down,stars,best_score,text
0,0,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2G3aneMRgRMZwXqIHmSdXSG1hEM,u_9iFLIhMa8QaG,Jeri326,1,1665619889,0,0,0,5,527,"I tweaked it a little, removed onions because ..."
1,1,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2FsPC83HtzCsQAtOxlbL6RcaPbY,u_Lu6p25tmE77j,Mark467,50,1665277687,0,7,0,5,724,Bush used to have a white chili bean and it ma...
2,2,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2FPrSGyTv7PQkZq37j92r9mYGkP,u_s0LwgpZ8Jsqq,Barbara566,10,1664404557,0,3,0,5,710,I have a very complicated white chicken chili ...
3,3,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2DzdSIgV9qNiuBaLoZ7JQaartoC,u_fqrybAdYjgjG,jeansch123,1,1661787808,2,2,0,0,581,"In your introduction, you mentioned cream chee..."
4,4,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2DtZJuRQYeTFwXBoZRfRhBPEXjI,u_XXWKwVhKZD69,camper77,10,1664913823,1,7,0,0,820,Wonderful! I made this for a &#34;Chili/Stew&#...
5,5,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2Do918IutExN0pWEOFMU4cbiT8v,u_BALTQJIvWtYr,nikhita,1,1661354351,0,3,1,5,518,amazing! my boyfriend loved it so much! going ...
6,6,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_24hhcbywpsgGqG7yeDFH1IPZCb8,u_HuJVXMzQqJoI,Sandy1256,1,1644088805,0,11,0,5,833,Wow!!! This recipe is excellent as written!! ...
7,7,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_24cuIEu6UoJvd5VRMAoqnTPR9wd,u_uj79hCc4xVhm,Towanka,1,1643942114,0,28,2,0,891,This is delicious and I make it often. One suc...
8,8,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_24cc4VZR87RR5vdkYAnsnq6YaOS,u_xDTU4BqIVIc9,Quest,1,1643933124,0,6,0,5,693,I absolutely love this recipe. I&#39;ve tweake...
9,9,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_24Fso1ApihQ91uCmNvhBDDO1k2P,u_cDoX9ujcQEoc,Susannah953,1,1643237839,0,0,0,5,404,I make this a lot … my kids and there friends ...


In [26]:
reviews_df.describe()

,Unnamed: 0,recipe_number,recipe_code,user_reputation,created_at,reply_count,thumbs_up,thumbs_down,stars,best_score
count,18182.000000,18182.000000,18182.000000,18182.000000,1.818200e+04,18182.000000,18182.000000,18182.000000,18182.000000,18182.000000
mean,121.465295,38.689363,21773.667253,2.159608,1.623710e+09,0.014630,1.089264,0.549335,4.288802,153.162138
std,116.747893,29.786647,23965.109637,10.014666,5.468697e+06,0.137974,4.201004,3.470124,1.544786,141.075316
min,0.000000,1.000000,386.000000,0.000000,1.613035e+09,0.000000,0.000000,0.000000,0.000000,0.000000
25%,45.000000,12.000000,6086.000000,1.000000,1.622717e+09,0.000000,0.000000,0.000000,5.000000,100.000000
50%,91.000000,33.000000,14600.000000,1.000000,1.622718e+09,0.000000,0.000000,0.000000,5.000000,100.000000
75%,150.000000,64.000000,33121.000000,1.000000,1.622718e+09,0.000000,0.000000,0.000000,5.000000,100.000000
max,724.000000,100.000000,191775.000000,520.000000,1.665756e+09,3.000000,106.000000,126.000000,5.000000,946.000000


In [27]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18182 entries, 0 to 18181
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       18182 non-null  int64 
 1   recipe_number    18182 non-null  int64 
 2   recipe_code      18182 non-null  int64 
 3   recipe_name      18182 non-null  object
 4   comment_id       18182 non-null  object
 5   user_id          18182 non-null  object
 6   user_name        18182 non-null  object
 7   user_reputation  18182 non-null  int64 
 8   created_at       18182 non-null  int64 
 9   reply_count      18182 non-null  int64 
 10  thumbs_up        18182 non-null  int64 
 11  thumbs_down      18182 non-null  int64 
 12  stars            18182 non-null  int64 
 13  best_score       18182 non-null  int64 
 14  text             18180 non-null  object
dtypes: int64(10), object(5)
memory usage: 2.1+ MB


In [28]:
reviews_df.isna().sum()

Unnamed: 0         0
recipe_number      0
recipe_code        0
recipe_name        0
comment_id         0
user_id            0
user_name          0
user_reputation    0
created_at         0
reply_count        0
thumbs_up          0
thumbs_down        0
stars              0
best_score         0
text               2
dtype: int64

In [29]:
reviews_df[reviews_df['text'].isna() == True]

,Unnamed: 0,recipe_number,recipe_code,recipe_name,comment_id,user_id,user_name,user_reputation,created_at,reply_count,thumbs_up,thumbs_down,stars,best_score,text
1507,344,3,2832,Cheeseburger Soup,sp_aUSaElGf_2832_c_260955,u_1oKZmfyycO3xVnQeffIKHuaKT2U,No.1FamilyBaker,1,1622717645,0,0,0,5,100,NaN
2722,299,6,21444,Favorite Chicken Potpie,sp_aUSaElGf_21444_c_260817,u_1oKXz42APTrXTFF6SGvR3YMBJGy,TRGriggs,1,1622717652,0,0,0,5,100,NaN


In [30]:
reviews_df.dropna(inplace = True)

In [31]:
reviews_df.isna().sum()

Unnamed: 0         0
recipe_number      0
recipe_code        0
recipe_name        0
comment_id         0
user_id            0
user_name          0
user_reputation    0
created_at         0
reply_count        0
thumbs_up          0
thumbs_down        0
stars              0
best_score         0
text               0
dtype: int64

In [32]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18180 entries, 0 to 18181
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       18180 non-null  int64 
 1   recipe_number    18180 non-null  int64 
 2   recipe_code      18180 non-null  int64 
 3   recipe_name      18180 non-null  object
 4   comment_id       18180 non-null  object
 5   user_id          18180 non-null  object
 6   user_name        18180 non-null  object
 7   user_reputation  18180 non-null  int64 
 8   created_at       18180 non-null  int64 
 9   reply_count      18180 non-null  int64 
 10  thumbs_up        18180 non-null  int64 
 11  thumbs_down      18180 non-null  int64 
 12  stars            18180 non-null  int64 
 13  best_score       18180 non-null  int64 
 14  text             18180 non-null  object
dtypes: int64(10), object(5)
memory usage: 2.2+ MB


In [34]:
reviews_df['created_at']

0        1665619889
1        1665277687
2        1664404557
3        1661787808
4        1664913823
            ...    
18177    1622717977
18178    1613036720
18179    1622717844
18180    1622717233
18181    1622717625
Name: created_at, Length: 18180, dtype: int64

In [35]:
date_format = '%d%M%Y'
reviews_df['created_at'] = pd.to_datetime(reviews_df['created_at'], unit='s').dt.strftime('%d-%m-%Y')

In [36]:
reviews_df['created_at']

0        13-10-2022
1        09-10-2022
2        28-09-2022
3        29-08-2022
4        04-10-2022
            ...    
18177    03-06-2021
18178    11-02-2021
18179    03-06-2021
18180    03-06-2021
18181    03-06-2021
Name: created_at, Length: 18180, dtype: object

In [38]:
reviews_df.drop(columns = ['Unnamed: 0'], inplace = True)

In [39]:
reviews_df.head(10)

,recipe_number,recipe_code,recipe_name,comment_id,user_id,user_name,user_reputation,created_at,reply_count,thumbs_up,thumbs_down,stars,best_score,text
0,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2G3aneMRgRMZwXqIHmSdXSG1hEM,u_9iFLIhMa8QaG,Jeri326,1,13-10-2022,0,0,0,5,527,"I tweaked it a little, removed onions because ..."
1,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2FsPC83HtzCsQAtOxlbL6RcaPbY,u_Lu6p25tmE77j,Mark467,50,09-10-2022,0,7,0,5,724,Bush used to have a white chili bean and it ma...
2,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2FPrSGyTv7PQkZq37j92r9mYGkP,u_s0LwgpZ8Jsqq,Barbara566,10,28-09-2022,0,3,0,5,710,I have a very complicated white chicken chili ...
3,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2DzdSIgV9qNiuBaLoZ7JQaartoC,u_fqrybAdYjgjG,jeansch123,1,29-08-2022,2,2,0,0,581,"In your introduction, you mentioned cream chee..."
4,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2DtZJuRQYeTFwXBoZRfRhBPEXjI,u_XXWKwVhKZD69,camper77,10,04-10-2022,1,7,0,0,820,Wonderful! I made this for a &#34;Chili/Stew&#...
5,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2Do918IutExN0pWEOFMU4cbiT8v,u_BALTQJIvWtYr,nikhita,1,24-08-2022,0,3,1,5,518,amazing! my boyfriend loved it so much! going ...
6,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_24hhcbywpsgGqG7yeDFH1IPZCb8,u_HuJVXMzQqJoI,Sandy1256,1,05-02-2022,0,11,0,5,833,Wow!!! This recipe is excellent as written!! ...
7,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_24cuIEu6UoJvd5VRMAoqnTPR9wd,u_uj79hCc4xVhm,Towanka,1,04-02-2022,0,28,2,0,891,This is delicious and I make it often. One suc...
8,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_24cc4VZR87RR5vdkYAnsnq6YaOS,u_xDTU4BqIVIc9,Quest,1,04-02-2022,0,6,0,5,693,I absolutely love this recipe. I&#39;ve tweake...
9,1,14299,Creamy White Chili,sp_aUSaElGf_14299_c_24Fso1ApihQ91uCmNvhBDDO1k2P,u_cDoX9ujcQEoc,Susannah953,1,26-01-2022,0,0,0,5,404,I make this a lot … my kids and there friends ...


In [40]:
reviews_df.describe()

,recipe_number,recipe_code,user_reputation,reply_count,thumbs_up,thumbs_down,stars,best_score
count,18180.000000,18180.000000,18180.000000,18180.000000,18180.000000,18180.000000,18180.000000,18180.000000
mean,38.693124,21774.727283,2.159736,0.014631,1.089384,0.549395,4.288724,153.167987
std,29.786123,23966.015985,10.015210,0.137981,4.201219,3.470310,1.544853,141.081974
min,1.000000,386.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12.000000,6086.000000,1.000000,0.000000,0.000000,0.000000,5.000000,100.000000
50%,33.000000,14600.000000,1.000000,0.000000,0.000000,0.000000,5.000000,100.000000
75%,64.000000,33121.000000,1.000000,0.000000,0.000000,0.000000,5.000000,100.000000
max,100.000000,191775.000000,520.000000,3.000000,106.000000,126.000000,5.000000,946.000000


In [41]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18180 entries, 0 to 18181
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   recipe_number    18180 non-null  int64 
 1   recipe_code      18180 non-null  int64 
 2   recipe_name      18180 non-null  object
 3   comment_id       18180 non-null  object
 4   user_id          18180 non-null  object
 5   user_name        18180 non-null  object
 6   user_reputation  18180 non-null  int64 
 7   created_at       18180 non-null  object
 8   reply_count      18180 non-null  int64 
 9   thumbs_up        18180 non-null  int64 
 10  thumbs_down      18180 non-null  int64 
 11  stars            18180 non-null  int64 
 12  best_score       18180 non-null  int64 
 13  text             18180 non-null  object
dtypes: int64(8), object(6)
memory usage: 2.1+ MB


In [43]:
# Data Preprocessing

# Convert to lowercase
reviews_df['text'] = reviews_df['text'].str.lower()  

# Remove punctuations
reviews_df['text'] = reviews_df['text'].str.replace(r'[^\w\s]', '', regex=True)  
texts = reviews_df['text'].values
labels = reviews_df['stars'].values

In [44]:
# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [45]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [46]:
# Tokenization and padding
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=100)
X_test_pad = pad_sequences(X_test_seq, maxlen=100)

In [47]:
# Model design
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [48]:
# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [51]:
# Train model
model.fit(X_train_pad, y_train, validation_split=0.2, epochs=10, batch_size=32)

Epoch 1/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.8672 - loss: 0.3928 - val_accuracy: 0.7580 - val_loss: 0.9101
Epoch 2/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.8876 - loss: 0.3238 - val_accuracy: 0.7329 - val_loss: 1.0169
Epoch 3/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.9023 - loss: 0.2794 - val_accuracy: 0.7563 - val_loss: 1.2050
Epoch 4/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.9140 - loss: 0.2372 - val_accuracy: 0.7401 - val_loss: 1.2446
Epoch 5/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.9263 - loss: 0.2189 - val_accuracy: 0.7480 - val_loss: 1.3920
Epoch 6/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.9316 - loss: 0.1999 - val_accuracy: 0.7398 - val_loss: 1.5870
Epoch 7/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.9380 - loss: 0.1786 - val_accuracy: 0.7401 - val_loss: 1.6769
Epoch 8/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.9407 - loss: 0.1711 - val_accu

In [52]:
# Evaluate model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7405 - loss: 1.8326
Test Accuracy: 0.74


In [101]:
from tensorflow.keras.utils import to_categorical

# One-hot encode labels for training and testing
y_train_one_hot = to_categorical(y_train, num_classes=10)
y_test_one_hot = to_categorical(y_test, num_classes=10)

In [102]:
# Random Normal Initializer with SGD
init = tf.keras.initializers.random_normal()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
model_normal_sgd = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(100,), kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(10, activation='softmax', kernel_initializer=tf.keras.initializers.glorot_normal())
])

In [103]:
model_normal_sgd.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_normal_sgd.fit(X_train_pad, y_train_one_hot, epochs=10, batch_size=160, validation_data=(X_test_pad, y_test_one_hot), verbose=1)

Epoch 1/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6569 - loss: 1.9546 - val_accuracy: 0.7563 - val_loss: 0.9105
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7594 - loss: 0.9132 - val_accuracy: 0.7563 - val_loss: 0.8928
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7546 - loss: 0.8971 - val_accuracy: 0.7563 - val_loss: 0.8857
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7609 - loss: 0.8733 - val_accuracy: 0.7563 - val_loss: 0.8804
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7624 - loss: 0.8662 - val_accuracy: 0.7561 - val_loss: 0.8784
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7619 - loss: 0.8659 - val_accuracy: 0.7563 - val_loss: 0.9021
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7621 - loss: 0.8690 - val_accuracy: 0.7563 - val_loss: 0.8784
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7648 - loss: 0.8541 - val_accuracy: 0.7558 - val_loss:

In [104]:
# Train and Test Accuracy
train_acc_normal_sgd = model_normal_sgd.evaluate(X_train_pad, y_train_one_hot, verbose=0)[1]
test_acc_normal_sgd = model_normal_sgd.evaluate(X_test_pad, y_test_one_hot, verbose=0)[1]
print('Train Accuracy (Random Normal Init with SGD):', train_acc_normal_sgd)
print('Test Accuracy (Random Normal Init with SGD):', test_acc_normal_sgd)

Train Accuracy (Random Normal Init with SGD): 0.7623762488365173
Test Accuracy (Random Normal Init with SGD): 0.7563256621360779


In [105]:
# Glorot Uniform Initializer with SGD
init = tf.keras.initializers.glorot_uniform()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
model_uniform_sgd = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(100,), kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(10, activation='softmax', kernel_initializer=tf.keras.initializers.glorot_normal())
])

In [106]:
model_uniform_sgd.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_uniform_sgd.fit(X_train_pad, y_train_one_hot, epochs=10, batch_size=160, validation_data=(X_test_pad, y_test_one_hot), verbose=1)

Epoch 1/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6129 - loss: 9324843296359626950823641088.0000 - val_accuracy: 0.7563 - val_loss: 1.0469
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7573 - loss: 1.0199 - val_accuracy: 0.7563 - val_loss: 0.9740
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7611 - loss: 0.9545 - val_accuracy: 0.7563 - val_loss: 0.9425
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7630 - loss: 0.9251 - val_accuracy: 0.7563 - val_loss: 0.9250
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7685 - loss: 0.8957 - val_accuracy: 0.7563 - val_loss: 0.9143
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7617 - loss: 0.9039 - val_accuracy: 0.7563 - val_loss: 0.9074
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7643 - loss: 0.8888 - val_accuracy: 0.7563 - val_loss: 0.9026
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7663 - loss: 0.8862 - val_a

In [107]:
# Train and Test Accuracy
train_acc_uniform_sgd = model_uniform_sgd.evaluate(X_train_pad, y_train_one_hot, verbose=0)[1]
test_acc_uniform_sgd = model_uniform_sgd.evaluate(X_test_pad, y_test_one_hot, verbose=0)[1]
print('Train Accuracy (Glorot Uniform Init with SGD):', train_acc_uniform_sgd)
print('Test Accuracy (Glorot Uniform Init with SGD):', test_acc_uniform_sgd)

Train Accuracy (Glorot Uniform Init with SGD): 0.7616199254989624
Test Accuracy (Glorot Uniform Init with SGD): 0.7563256621360779


In [108]:
# He Uniform Initializer with SGD
init = tf.keras.initializers.he_uniform()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
model_he_sgd = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(100,), kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(10, activation='softmax', kernel_initializer=tf.keras.initializers.glorot_normal())
])

In [109]:
model_he_sgd.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_he_sgd.fit(X_train_pad, y_train_one_hot, epochs=10, batch_size=160, validation_data=(X_test_pad, y_test_one_hot), verbose=1)

Epoch 1/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7067 - loss: 5538698.0000 - val_accuracy: 0.7563 - val_loss: 0.9648
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7641 - loss: 0.9675 - val_accuracy: 0.7563 - val_loss: 0.9386
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7611 - loss: 0.9437 - val_accuracy: 0.7563 - val_loss: 0.9369
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7638 - loss: 0.9160 - val_accuracy: 0.7563 - val_loss: 0.9007
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7623 - loss: 0.8963 - val_accuracy: 0.7563 - val_loss: 0.8940
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7582 - loss: 0.8989 - val_accuracy: 0.7563 - val_loss: 0.9057
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7598 - loss: 0.8858 - val_accuracy: 0.7563 - val_loss: 0.8824
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7597 - loss: 0.8829 - val_accuracy: 0.7563 - val

In [110]:
# Train and Test Accuracy
train_acc_he_sgd = model_he_sgd.evaluate(X_train_pad, y_train_one_hot, verbose=0)[1]
test_acc_he_sgd = model_he_sgd.evaluate(X_test_pad, y_test_one_hot, verbose=0)[1]
print('Train Accuracy (He Uniform Init with SGD):', train_acc_he_sgd)
print('Test Accuracy (He Uniform Init with SGD):', test_acc_he_sgd)

Train Accuracy (He Uniform Init with SGD): 0.7616199254989624
Test Accuracy (He Uniform Init with SGD): 0.7563256621360779


In [111]:
# He Uniform Initializer with Adagrad
init = tf.keras.initializers.he_uniform()
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.1)
model_he_adagrad = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(100,), kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(10, activation='softmax', kernel_initializer=tf.keras.initializers.glorot_normal())
])

In [112]:
model_he_adagrad.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_he_adagrad.fit(X_train_pad, y_train_one_hot, epochs=10, batch_size=160, validation_data=(X_test_pad, y_test_one_hot), verbose=1)

Epoch 1/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6545 - loss: 187.8781 - val_accuracy: 0.7478 - val_loss: 1.1099
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7542 - loss: 1.1185 - val_accuracy: 0.7522 - val_loss: 0.9827
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7604 - loss: 0.9434 - val_accuracy: 0.7503 - val_loss: 0.9811
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7646 - loss: 0.8993 - val_accuracy: 0.7528 - val_loss: 0.9457
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7591 - loss: 0.8889 - val_accuracy: 0.7517 - val_loss: 0.9359
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7645 - loss: 0.8731 - val_accuracy: 0.7525 - val_loss: 0.9298
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7660 - loss: 0.8612 - val_accuracy: 0.7508 - val_loss: 0.9300
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7673 - loss: 0.8681 - val_accuracy: 0.7528 - val_los

In [113]:
# Train and Test Accuracy
train_acc_he_adagrad = model_he_adagrad.evaluate(X_train_pad, y_train_one_hot, verbose=0)[1]
test_acc_he_adagrad = model_he_adagrad.evaluate(X_test_pad, y_test_one_hot, verbose=0)[1]
print('Train Accuracy (He Uniform Init with Adagrad):', train_acc_he_adagrad)
print('Test Accuracy (He Uniform Init with Adagrad):', test_acc_he_adagrad)

Train Accuracy (He Uniform Init with Adagrad): 0.7627200484275818
Test Accuracy (He Uniform Init with Adagrad): 0.7530252933502197


In [114]:
# He Uniform Initializer with Adam
init = tf.keras.initializers.he_uniform()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_he_adam = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(100,), kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(10, activation='softmax', kernel_initializer=tf.keras.initializers.glorot_normal())
])

In [115]:
model_he_adam.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_he_adam.fit(X_train_pad, y_train_one_hot, epochs=10, batch_size=160, validation_data=(X_test_pad, y_test_one_hot), verbose=1)

Epoch 1/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.3297 - loss: 280.7452 - val_accuracy: 0.5729 - val_loss: 32.2313
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5842 - loss: 30.7432 - val_accuracy: 0.5734 - val_loss: 16.6110
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5963 - loss: 16.3812 - val_accuracy: 0.6059 - val_loss: 10.0740
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6233 - loss: 9.3954 - val_accuracy: 0.6221 - val_loss: 6.9236
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6333 - loss: 6.1835 - val_accuracy: 0.6496 - val_loss: 5.0523
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6467 - loss: 4.2517 - val_accuracy: 0.6326 - val_loss: 3.9141
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6536 - loss: 3.2342 - val_accuracy: 0.6771 - val_loss: 3.1672
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6739 - loss: 2.5157 - val_accuracy: 0.6785 - va

In [116]:
# Train and Test Accuracy
train_acc_he_adam = model_he_adam.evaluate(X_train_pad, y_train_one_hot, verbose=0)[1]
test_acc_he_adam = model_he_adam.evaluate(X_test_pad, y_test_one_hot, verbose=0)[1]
print('Train Accuracy (He Uniform Init with Adam, lr=0.001):', train_acc_he_adam)
print('Test Accuracy (He Uniform Init with Adam, lr=0.001):', test_acc_he_adam)

Train Accuracy (He Uniform Init with Adam, lr=0.001): 0.7194032073020935
Test Accuracy (He Uniform Init with Adam, lr=0.001): 0.7079207897186279


In [117]:
# He Uniform Initializer with Adam and Early Stopping
init = tf.keras.initializers.he_uniform()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model_he_adam_early = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(100,), kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(32, activation='relu', kernel_initializer=init),
    layers.Dense(10, activation='softmax', kernel_initializer=tf.keras.initializers.glorot_normal())
])

In [118]:
model_he_adam_early.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True, verbose=1)

# Train with early stopping
history = model_he_adam_early.fit(X_train_pad, y_train_one_hot, epochs=2000, callbacks=[es], validation_data=(X_test_pad, y_test_one_hot), verbose=1)


Epoch 1/2000
455/455 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6441 - loss: 34.1510 - val_accuracy: 0.7558 - val_loss: 0.9864
Epoch 2/2000
455/455 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7546 - loss: 0.9557 - val_accuracy: 0.7522 - val_loss: 0.8918
Epoch 3/2000
455/455 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7538 - loss: 0.9013 - val_accuracy: 0.7550 - val_loss: 0.8893
Epoch 4/2000
455/455 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7673 - loss: 0.8710 - val_accuracy: 0.7561 - val_loss: 0.8793
Epoch 5/2000
455/455 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7591 - loss: 0.8742 - val_accuracy: 0.7563 - val_loss: 0.8722
Epoch 6/2000
455/455 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7646 - loss: 0.8643 - val_accuracy: 0.7563 - val_loss: 0.8719
Epoch 7/2000
455/455 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7618 - loss: 0.8648 - val_accuracy: 0.7563 - val_loss: 0.8713
Epoch 8/2000
455/455 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7545 - loss: 0.8786 -

In [119]:
# Train and Test Accuracy
train_acc_he_adam_early = model_he_adam_early.evaluate(X_train_pad, y_train_one_hot, verbose=0)[1]
test_acc_he_adam_early = model_he_adam_early.evaluate(X_test_pad, y_test_one_hot, verbose=0)[1]
print('Train Accuracy (He Uniform Init with Adam, lr=0.01 and Early Stopping):', train_acc_he_adam_early)
print('Test Accuracy (He Uniform Init with Adam, lr=0.01 and Early Stopping):', test_acc_he_adam_early)

Train Accuracy (He Uniform Init with Adam, lr=0.01 and Early Stopping): 0.7616199254989624
Test Accuracy (He Uniform Init with Adam, lr=0.01 and Early Stopping): 0.7563256621360779
